In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from pandas import DataFrame as df
import plotly.express as px
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import analyse_file
from EC_data_processing_lib import get_decimal_separator
from Data_setup_Scotita import data_set
import os
from galvani import BioLogic
fig = go.Figure()
colors = px.colors.qualitative.Safe

for i in data_set:
    file = data_set[i]['data_CA']
    if not os.path.exists(file):
        continue
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        data = pd.read_csv(file, encoding='ansi', sep="\t", decimal=get_decimal_separator(file),  skiprows=range(headerlength))
        current_column = '<I>/mA'
    else:
        mpr_file = BioLogic.MPRfile(file)
        data = pd.DataFrame(mpr_file.data)
        current_column = 'I/mA'
    current = data[current_column]
    current_mass = current / data_set[i]['mass']
    time = data['time/s']/60  #converts seconds to minutes
    time = time - time[0] #sets intitial time to 0 

    j = 0
    Final_Values = []
    # calculates the final current values and prints them along with standard deviation    
    
    time_start_last_10percent = float(time.tail(1)*0.95)
    time_last_10percent_mask = time > time_start_last_10percent
    final_J = df.mean(current_mass[time_last_10percent_mask])
    final_J = np.round(final_J,3)
    final_J_error = df.std(current_mass[time_last_10percent_mask])
    final_J_error = np.round(final_J_error,3)

    print(data_set[i]['label'], str(final_J), str(final_J_error))   

    fig.add_trace(go.Scatter(
        x=time,
        y=current_mass,
        name= data_set[i]['label'],
        line_color = colors[data_set[i]['color_index']],
    ))
fig.update_xaxes(range=[0.00, 120],
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[0, 1],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
fig.update_layout(
    #title="CV",
    xaxis_title="Time (min)",
    yaxis_title="Current (mA/cm<sup>2</sup>/mg)",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
        
    )
)

fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(0, 0, 0, 0)', #sets the background to be transparent... might appear black in .jpeg but will be transparent in the .svg
})
fig.update_layout(
    width = 500,
    height = 500)
fig.show()
fig.write_image('Chronoamperometry.jpeg')
fig.write_image('Chronoamperometry.svg')

